# Reconocimiento de dígitos con KNN



Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [ ]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import metnum as mt
import datetime
import os
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from numpy import savetxt
from sklearn.metrics import accuracy_score

#percentage over total of train cases
PERCENTAGE_OF_TRAIN_CASES = 0.8
#neighbors for finding the mode in KNN
N_NEIGHBORS = 100
#components for PCA
N_COMPONENTS=40
#number of iterations to find eigenvalues and eigenvectors in power iteration
N_ITERATIONS=5000
#epsilon for power iteration
EPSILON=1e-10
#directory for saving matrix files
timestamp = datetime.now().strftime("%m_%d_%H_%M_%S")
DIRECTORY_NAME="{}".format(timestamp)
os.makedirs(DIRECTORY_NAME+"/", exist_ok=True)

%load_ext autoreload
%autoreload 2

df_train = pd.read_csv("../data/train.csv")

TOTAL_TRAIN_CASES = int(PERCENTAGE_OF_TRAIN_CASES*len(df_train))

#shuffle the train cases.
df_train = df_train[0: TOTAL_TRAIN_CASES].sample(frac=1)

# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

limit = int(0.8 * X.shape[0]) 

X_train, y_train = X[:limit], y[:limit]
X_val, y_val = X[limit:], y[limit:]

assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")

### Opcional: guardamos las matrices de entrenamiento en un archivo si queremos reproducir el experimento

In [ ]:
savetxt('{}/X_train.csv'.format(DIRECTORY_NAME), X_train, delimiter=',')
savetxt('{}/y_train.csv'.format(DIRECTORY_NAME), y_train, delimiter=',')
savetxt('{}/X_val.csv'.format(DIRECTORY_NAME), X_val, delimiter=',')
savetxt('{}/y_val.csv'.format(DIRECTORY_NAME), y_val, delimiter=',')

## Testing sin PCA (raw data)

### Entrenamos el dato con los test cases sin modificar y probamos su accuracy

In [ ]:
%%time

clf = mt.KNNClassifier(N_NEIGHBORS)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val[0:])
acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))


### Ejemplo para probar la capacidad del modelo a mano.

In [ ]:
random_example= 598

img = X_val[random_example].reshape(28, 28)
plt.imshow(img, cmap="Greys")

print("Prediction: {} - Digit: {}".format( int(y_pred[random_example]), int(y_val[random_example])))

## Testing con PCA

### Usamos PCA para transformar la data

In [ ]:
%%time
pca = mt.PCA(N_COMPONENTS, N_ITERATIONS, EPSILON)
pca.fit(X_train)

X_train_pca, X_val_pca = pca.transform(X_train), pca.transform(X_val)


### Opcional: guardamos las matrices de entrenamiento en un archivo si queremos reproducir el experimento

In [ ]:
savetxt('{}/X_train_pca.csv'.format(DIRECTORY_NAME), X_train_pca, delimiter=',')
savetxt('{}/X_val_pca.csv'.format(DIRECTORY_NAME), X_val_pca, delimiter=',')

### Entrenamos el modelo con la data de PCA y testeamos accuracy

In [ ]:
%%time
clf = mt.KNNClassifier(N_NEIGHBORS)

clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_val_pca[0:])
acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))

# Testeo automático moviendo parámetros

In [4]:
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import metnum as mt
import datetime
import os
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from numpy import savetxt
from sklearn.metrics import *
from IPython.display import display, HTML

def calculate_metrics(y_val, y_pred, time):
    
    acc = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average=None)
    recall = recall_score(y_val, y_pred, average=None)
    f1 = f1_score(y_val, y_pred, average=None)
    kappa = -1#kappa_score(y_val, y_pred, average=None)
    
    return (acc, precision, recall, 
            f1, kappa, time)

def experiment_generic(variables, pca):
    
    for variable in variables:
        name = variable["name"]
        value = variable["value"]
        if(name=="PERCENTAGE_OF_TRAIN_CASES"):
            PERCENTAGE_OF_TRAIN_CASES = value
        elif (name=="N_NEIGHBORS"):
            N_NEIGHBORS = value
        elif (name=="N_COMPONENTS"):
            N_COMPONENTS = value
        elif (name == "N_ITERATIONS"):
            N_ITERATIONS = value
        elif(name == "EPSILON"):
            EPSILON = value
            
    print("{} : {}\n{} : {}\n{} : {}\n{} : {}\n{} : {}\n".format("PERCENTAGE_OF_TRAIN_CASES",PERCENTAGE_OF_TRAIN_CASES,
                                                                 "N_NEIGHBORS",N_NEIGHBORS,
                                                                 "N_COMPONENTS",N_COMPONENTS,
                                                                 "N_ITERATIONS",N_ITERATIONS,
                                                                 "EPSILON",EPSILON))
    return experiment(PERCENTAGE_OF_TRAIN_CASES, 
                   N_NEIGHBORS, 
                   N_COMPONENTS, 
                   N_ITERATIONS, 
                   EPSILON,
                   pca)
    
    
def experiment(PERCENTAGE_OF_TRAIN_CASES, 
                   N_NEIGHBORS, 
                   N_COMPONENTS, 
                   N_ITERATIONS, 
                   EPSILON,
                   pca):
    
    start = time.time()
    df_train = pd.read_csv("../data/train.csv")

    TOTAL_TRAIN_CASES = int(PERCENTAGE_OF_TRAIN_CASES*len(df_train))
    #shuffle the train cases.
    df_train = df_train[0: TOTAL_TRAIN_CASES].sample(frac=1)

    # Uso values para mandar todo a arrays de numpy
    X = df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)

    limit = int(0.8 * X.shape[0]) 

    X_train, y_train = X[:limit], y[:limit]
    X_val, y_val = X[limit:], y[limit:]
    
    if(pca):
        pca = mt.PCA(N_COMPONENTS, N_ITERATIONS, EPSILON)
        pca.fit(X_train)

        X_train, X_val = pca.transform(X_train), pca.transform(X_val)

    
    clf = mt.KNNClassifier(N_NEIGHBORS)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val[0:])
    
    return calculate_metrics(y_val, y_pred, time.time()-start)


def create_df(variable, experiment_results):
    df = pd.DataFrame(columns=[variable, 'ACCURACY', 'TIME'])
    
    for i in range(10):
        df['PRECISION_'+str(i)] = np.NaN
    for i in range(10):
        df['RECALL_'+str(i)] = np.NaN
    for i in range(10):
        df['F1_'+str(i)] = np.NaN
        
    return df.append(experiment_results, sort=False)


def experiment_generic_move_variable(variable_range, fixed_variables, pca=1):
    
    timestamp = datetime.now().strftime("%m-%d-%H-%M-%S")
    
    experiment_results = []
    
    NAME_OF_VARIABLE = variable_range["name"]
    print("variable range: "+str(variable_range)+"\nFixed variables:"+str(fixed_variables))
    
    for value in variable_range["range"]:
        fixed_variables.append({"name":NAME_OF_VARIABLE, "value":value})
        
        (acc, precision, recall, 
        f1, kappa, execution_time) =  experiment_generic(fixed_variables, pca)
        
        data = {
            NAME_OF_VARIABLE : value,
            'ACCURACY' : acc,
            'TIME' : round(execution_time)
        }
        
        for i in range(10):
            data['PRECISION_'+str(i)] = precision[i]
            data['RECALL_'+str(i)] = recall[i]
            data['F1_'+str(i)] = f1[i]
            
        experiment_results.append(data)
        
    filename='test_'+NAME_OF_VARIABLE+'-{}'.format(time.time())
    
    df_result = create_df(NAME_OF_VARIABLE, experiment_results)  
    df_result.to_csv(filename+'.csv', encoding='utf-8', index=False)
    
    metadata = {
        'TIMESTAMP' : [timestamp]
    }
    
    metadata_columns = ['TIMESTAMP']
    
    for fixed_variable in fixed_variables:
        metadata[fixed_variable["name"]] = fixed_variable["value"]
        metadata_columns.append(fixed_variable["name"])
                
    df_metadata = pd.DataFrame(metadata, columns = metadata_columns)
    df_metadata.to_csv(filename+'-metadata.csv',encoding = 'utf-8', index=False)
    
    display(df_metadata)
    display(df_result)

        
def main():
    #percentage over total of train cases
    PERCENTAGE_OF_TRAIN_CASES = 0.5
    #neighbors for finding the mode in KNN
    N_NEIGHBORS = 20
    #components for PCA
    N_COMPONENTS=30
    #number of iterations to find eigenvalues and eigenvectors in power iteration
    N_ITERATIONS=2000
    #epsilon for power iteration
    EPSILON=1e-10

    
    experiment_generic_move_variable({"range" : np.arange(10000,22001,1000), 
                                      "name" : "N_NEIGHBORS"},
                                    [{"name":"N_COMPONENTS","value":N_COMPONENTS},
                                     {"name":"PERCENTAGE_OF_TRAIN_CASES","value":PERCENTAGE_OF_TRAIN_CASES},
                                     {"name":"N_ITERATIONS","value":N_ITERATIONS},
                                     {"name":"EPSILON","value":EPSILON}],1)
    
#     experiment_generic_move_variable({"range" : np.arange(1,30,1), 
#                                       "name" : "N_COMPONENTS"},
#                                     [{"name":"N_NEIGHBORS","value":N_NEIGHBORS},
#                                      {"name":"PERCENTAGE_OF_TRAIN_CASES","value":PERCENTAGE_OF_TRAIN_CASES},
#                                      {"name":"N_ITERATIONS","value":N_ITERATIONS},
#                                      {"name":"EPSILON","value":EPSILON}],1)
    
#     experiment_generic_move_variable({"range" : np.arange(0.1,1,0.1), 
#                                       "name" : "PERCENTAGE_OF_TRAIN_CASES"},
#                                     [{"name":"N_NEIGHBORS","value":N_NEIGHBORS},
#                                      {"name":"N_COMPONENTS","value":N_COMPONENTS},
#                                      {"name":"N_ITERATIONS","value":N_ITERATIONS},
#                                      {"name":"EPSILON","value":EPSILON}],1)
        
#     experiment_generic_move_variable({"range" : np.arange(100,2000,100), 
#                                       "name" : "N_ITERATIONS"},
#                                     [{"name":"N_NEIGHBORS","value":N_NEIGHBORS},
#                                      {"name":"PERCENTAGE_OF_TRAIN_CASES","value":PERCENTAGE_OF_TRAIN_CASES},
#                                      {"name":"N_COMPONENTS","value":N_COMPONENTS},
#                                      {"name":"EPSILON","value":EPSILON}],1)
        
#     experiment_generic_move_variable({"range" : np.arange(5,7,1), 
#                                       "name" : "EPSILON"},
#                                     [{"name":"N_NEIGHBORS","value":N_NEIGHBORS},
#                                      {"name":"PERCENTAGE_OF_TRAIN_CASES","value":PERCENTAGE_OF_TRAIN_CASES},
#                                      {"name":"N_ITERATIONS","value":N_ITERATIONS},
#                                      {"name":"N_COMPONENTS","value":N_COMPONENTS}])
    

main()


variable range: {'range': array([10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000,
       19000, 20000, 21000, 22000]), 'name': 'N_NEIGHBORS'}
Fixed variables:[{'name': 'N_COMPONENTS', 'value': 30}, {'name': 'PERCENTAGE_OF_TRAIN_CASES', 'value': 0.5}, {'name': 'N_ITERATIONS', 'value': 2000}, {'name': 'EPSILON', 'value': 1e-10}]
PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 10000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 11000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 12000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 13000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10



/home/jazzer/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jazzer/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 14000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 15000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 16000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 17000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 18000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 19000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 20000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 21000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10

PERCENTAGE_OF_TRAIN_CASES : 0.5
N_NEIGHBORS : 22000
N_COMPONENTS : 30
N_ITERATIONS : 2000
EPSILON : 1e-10



,TIMESTAMP,N_COMPONENTS,PERCENTAGE_OF_TRAIN_CASES,N_ITERATIONS,EPSILON,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS,N_NEIGHBORS
0,06-25-00-15-40,30,0.5,2000,1.000000e-10,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000,22000


,N_NEIGHBORS,ACCURACY,TIME,PRECISION_0,PRECISION_1,PRECISION_2,PRECISION_3,PRECISION_4,PRECISION_5,PRECISION_6,...,F1_0,F1_1,F1_2,F1_3,F1_4,F1_5,F1_6,F1_7,F1_8,F1_9
0,10000,0.441190,15,0.976562,0.231539,0.852349,0.410072,0.655629,0.0,0.775665,...,0.756430,0.375870,0.468635,0.308108,0.557746,0.0,0.584527,0.615029,0.360465,0.341753
1,11000,0.349524,15,0.990050,0.187500,0.896552,0.307432,0.664596,0.0,0.668016,...,0.655684,0.315679,0.289425,0.254902,0.364566,0.0,0.497738,0.523748,0.184486,0.329949
2,12000,0.291429,15,0.984694,0.177661,0.725275,0.271493,0.582781,0.0,0.853211,...,0.617600,0.301719,0.255814,0.176991,0.312611,0.0,0.320690,0.414376,0.104308,0.114391
3,13000,0.251429,14,0.978873,0.161664,0.640000,0.219231,0.688172,0.0,0.730496,...,0.516729,0.278251,0.201681,0.166911,0.246154,0.0,0.369838,0.332219,0.077273,0.020408
4,14000,0.161429,14,0.971831,0.124855,0.750000,0.101124,0.565217,0.0,0.652174,...,0.279352,0.221993,0.064240,0.057878,0.113290,0.0,0.184049,0.157303,0.035714,0.044625
5,15000,0.128810,14,1.000000,0.121493,0.181818,0.076923,0.210526,0.0,0.555556,...,0.106236,0.216663,0.009195,0.028829,0.018913,0.0,0.068807,0.087087,0.005236,0.000000
6,16000,0.113333,14,0.500000,0.115044,0.000000,0.055556,0.000000,0.0,0.000000,...,0.005195,0.206349,0.000000,0.012024,0.000000,0.0,0.000000,0.015267,0.000000,0.000000
7,17000,0.112381,14,0.000000,0.112381,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.202055,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,18000,0.112143,14,0.000000,0.112143,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.201670,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,19000,0.109524,15,0.000000,0.109524,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.197425,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
